#setup:
- connect to google drive in this step

In [ ]:
import pandas as pd
import math
import numpy as np
import os
import glob
import sklearn
import pickle
import glob
import scipy.io
import cv2
from google.colab.patches import cv2_imshow
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

In [ ]:
def get_data(individual, bodypart, h5_file):
  mouse_data = h5_file.xs(individual,level='individuals',axis=1)
  out_data = mouse_data.xs(bodypart,level='bodyparts',axis=1)
  out_data.columns = out_data.columns.droplevel("scorer")
  output = out_data.copy()
  return output

In [ ]:
def within_area(area_vector, input_coor):
  area_startx = area_vector[0]
  area_starty = area_vector[1]
  area_distx = area_vector[2]
  area_disty = area_vector[3]
  x = input_coor["x"].iloc[0]
  y = input_coor["y"].iloc[0]
  if (area_startx <= x <= (area_startx+area_distx)) and (area_starty <= y <= (area_starty+area_disty)):
    result = 1
  else:
    result = 0
  return result

In [ ]:
def euclid_dist(point1_coor, point2_coor):
  point1 = np.array((point1_coor["x"].iloc[0], point1_coor["y"].iloc[0]))
  point2 = np.array((point2_coor["x"].iloc[0], point2_coor["y"].iloc[0]))
  output_dist = np.linalg.norm(point1 - point2)
  return output_dist

In [ ]:
def euclid_angle(pointa_coor, pointb_coor, pointc_coor):
  #angle_{pointa, pointb, pointc}
  a = np.array((pointa_coor["x"].iloc[0], pointa_coor["y"].iloc[0]))
  b = np.array((pointb_coor["x"].iloc[0], pointb_coor["y"].iloc[0]))
  c = np.array((pointc_coor["x"].iloc[0], pointc_coor["y"].iloc[0]))

  ba = a - b
  bc = c - b

  ac_dot = np.dot(ba, bc)

  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
  angle = np.arccos(cosine_angle)

  result = np.degrees(angle)
  return result

#Open some corrected_behavior data:

In [ ]:
video_name = 'PZ71_1'

#corrected_behavior_file = open(output_dir + video_name + '_behavior.pickle', 'rb')
corrected_behavior_file = open(output_dir + video_name + '_corrected_behavior.pickle', 'rb')
corrected_behavior = pickle.load(corrected_behavior_file)
corrected_behavior_file.close()

# area_vec = [area_startx, area_starty, area_distx, area_disty]
female_side_mat = scipy.io.loadmat(project_path + "/videos/" + video_name + "_female_side.mat")
female_side_vec = female_side_mat['croprect'][0]
male_side_mat = scipy.io.loadmat(project_path + "/videos/" + video_name + "_male_side.mat")
male_side_vec = male_side_mat['croprect'][0]

h5_file = pd.read_hdf(project_path +'/videos/'+ video_name + 'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered.h5')
[nframes, ncols] = h5_file.shape

#open classifer model

In [ ]:
#Importing essential libraries
import matplotlib.pyplot as plt
from statistics import mean
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [ ]:
# load the model
classifier_file = open(output_dir +'classifer_1202v5.pickle', 'rb')
classifier = pickle.load(classifier_file)
classifier_file.close()
#scaler_file = open(output_dir +'classifer_1202v3_scaler.pickle', 'rb')
#scaler = pickle.load(scaler_file)
#scaler_file.close()

#analysis setup

In [ ]:
# get all data ready for accessing as needed later on
mouse1_feature_points = {}
mouse1_feature_points['snout'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['shoulder'] = get_data('mus1', 'shoulder', h5_file)
mouse1_feature_points['spine1'] = get_data('mus1', 'spine1', h5_file)
mouse1_feature_points['spine2'] = get_data('mus1', 'spine2', h5_file)
mouse1_feature_points['spine3'] = get_data('mus1', 'spine3', h5_file)
mouse1_feature_points['spine4'] = get_data('mus1', 'spine4', h5_file)
mouse1_feature_points['tailbase'] = get_data('mus1', 'tailbase', h5_file)

mouse2_feature_points = {}
mouse2_feature_points['snout'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['shoulder'] = get_data('mus2', 'shoulder', h5_file)
mouse2_feature_points['spine1'] = get_data('mus2', 'spine1', h5_file)
mouse2_feature_points['spine2'] = get_data('mus2', 'spine2', h5_file)
mouse2_feature_points['spine3'] = get_data('mus2', 'spine3', h5_file)
mouse2_feature_points['spine4'] = get_data('mus2', 'spine4', h5_file)
mouse2_feature_points['tailbase'] = get_data('mus2', 'tailbase', h5_file)

mouse3_feature_points = {}
mouse3_feature_points['snout'] = get_data('mus3', 'snout', h5_file)
mouse3_feature_points['shoulder'] = get_data('mus3', 'shoulder', h5_file)
mouse3_feature_points['spine1'] = get_data('mus3', 'spine1', h5_file)
mouse3_feature_points['spine2'] = get_data('mus3', 'spine2', h5_file)
mouse3_feature_points['spine3'] = get_data('mus3', 'spine3', h5_file)
mouse3_feature_points['spine4'] = get_data('mus3', 'spine4', h5_file)
mouse3_feature_points['tailbase'] = get_data('mus3', 'tailbase', h5_file)


In [ ]:
# male_side_vec = [x, y, width, height]; female_side_vec = [x, y, width, height]
relevant_area = female_side_vec
# relevant area is female_side_vec + male_side_vec size exactly right next to each other
relevant_area[2] = female_side_vec[2] + male_side_vec[2]
relevant_area[3] = female_side_vec[3] + male_side_vec[3]

In [ ]:
def torso_in_area(area_vector, mouse_num, i):
  # finds if any part of torso at all is in area
  if mouse_num == 1:
    feature_points = mouse1_feature_points
  elif mouse_num == 2:
    feature_points = mouse2_feature_points
  elif mouse_num == 3:
    feature_points = mouse3_feature_points

  if within_area(area_vector, feature_points['snout'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['shoulder'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine1'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine2'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine3'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine4'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['tailbase'].loc[[i]]):
    return 1
  else:
    return 0

In [ ]:
def snout_in_area(area_vector, mouse_num, i):
  # finds if any part of torso at all is in area
  if mouse_num == 1:
    feature_points = mouse1_feature_points
  elif mouse_num == 2:
    feature_points = mouse2_feature_points
  elif mouse_num == 3:
    feature_points = mouse3_feature_points

  if within_area(area_vector, feature_points['snout'].loc[[i]]):
    result = 1
  else:
    result = 0
  return result

In [ ]:
def shoulder_in_area(area_vector, mouse_num, i):
  # finds if any part of torso at all is in area
  if mouse_num == 1:
    feature_points = mouse1_feature_points
  elif mouse_num == 2:
    feature_points = mouse2_feature_points
  elif mouse_num == 3:
    feature_points = mouse3_feature_points

  if within_area(area_vector, feature_points['shoulder'].loc[[i]]):
    result = 1
  else:
    result = 0
  return result

In [ ]:
def get_i_features(i, input_features, first_mouse_feature_points, second_mouse_feature_points):
  col_num = 0

  if random.random() < 0.5:
    row_dists = [euclid_dist(first_mouse_point.loc[[i]], second_mouse_point.loc[[i]]) for first_mouse_point in list(first_mouse_feature_points.values()) for second_mouse_point in list(second_mouse_feature_points.values())]
  else:
    row_dists = [euclid_dist(first_mouse_point.loc[[i]], second_mouse_point.loc[[i]]) for second_mouse_point in list(second_mouse_feature_points.values()) for first_mouse_point in list(first_mouse_feature_points.values())]
  input_features[i:i+1, 0:49] = np.array(row_dists)
  col_num = 49

  for first_mouse_key, first_mouse_point in first_mouse_feature_points.items():
    if within_area(relevant_area, first_mouse_point.loc[[i]]) and within_area(relevant_area, second_mouse_feature_points[first_mouse_key].loc[[i]]):
      input_features[i, col_num] = 1
    else:
      input_features[i, col_num] = 0
    col_num = col_num + 1

  first_mouse_snout = first_mouse_feature_points['snout'].loc[[i]]
  first_mouse_shoulder = first_mouse_feature_points['shoulder'].loc[[i]]
  second_mouse_snout = second_mouse_feature_points['snout'].loc[[i]]
  second_mouse_shoulder = second_mouse_feature_points['shoulder'].loc[[i]]
  if snout_in_area(relevant_area, 1, i) and snout_in_area(relevant_area, 2, i) and shoulder_in_area(relevant_area, 1, i):
    angle_1 = euclid_angle(first_mouse_snout, first_mouse_shoulder, second_mouse_snout)
  else:
    angle_1 = -1
  if snout_in_area(relevant_area, 1, i) and snout_in_area(relevant_area, 2, i) and shoulder_in_area(relevant_area, 2, i):
    angle_2 = euclid_angle(second_mouse_snout, second_mouse_shoulder, first_mouse_snout)
  else:
    angle_2 = -1
  input_features[i, col_num] = max(angle_1, angle_2)
  col_num = col_num + 1
  input_features[i, col_num] = min(angle_1, angle_2)
  col_num = col_num + 1

  cross_dist1 = euclid_dist(first_mouse_feature_points['snout'].loc[[i]], second_mouse_feature_points['tailbase'].loc[[i]])
  cross_dist2 = euclid_dist(first_mouse_feature_points['tailbase'].loc[[i]], second_mouse_feature_points['snout'].loc[[i]])
  input_features[i, col_num] = abs(cross_dist1 - cross_dist2)
  col_num = col_num + 1

  return input_features

In [ ]:
def get_input_features(input_range):
  input_features = np.empty([input_range, 59])

  for i in range(0, input_range):
    if torso_in_area(relevant_area, 1, i) and torso_in_area(relevant_area, 2, i):
      first_mouse_feature_points = mouse1_feature_points
      second_mouse_feature_points = mouse2_feature_points
    elif torso_in_area(relevant_area, 1, i) and torso_in_area(relevant_area, 3, i):
      first_mouse_feature_points = mouse1_feature_points
      second_mouse_feature_points = mouse2_feature_points
    elif torso_in_area(relevant_area, 2, i) and torso_in_area(relevant_area, 3, i):
      first_mouse_feature_points = mouse2_feature_points
      second_mouse_feature_points = mouse3_feature_points
    
    input_features = get_i_features(i, input_features, first_mouse_feature_points, second_mouse_feature_points)

  input_features = np.nan_to_num(input_features, nan=-1)

  return input_features

# run analysis

In [ ]:
# should take 36 min+
forest_behavior = corrected_behavior.copy()
input_features = get_input_features(2000)
y_pred = classifier.predict(input_features)

for i in range(0, 2000):
  if corrected_behavior["behavior"][i] == 'mount':
    y_pred[i] = -1

for i in range(0, 2000):
  y_pred_range = y_pred[i:i+1]
  avg_y_pred = np.mean(y_pred_range)
  if avg_y_pred == -1:
    #print("No frames w 2 mice within 1s surround of: "+str(i))
    forest_behavior["behavior"][i] = 'unclear'
  elif avg_y_pred > 0.9:
    forest_behavior["behavior"][i] = 'forest_mount'

In [ ]:
with open(output_dir + video_name + '_forestv5_behavior.pickle', 'wb') as f:
    pickle.dump(forest_behavior, f)

#labelled forest behavior video

In [ ]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14/'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

end_frame = 6000
for video in [project_path + "/videos/PZ71_1.avi"]:
#for video in glob.glob(project_path + "/videos/" + "*.avi"):
  video_name = os.path.splitext(os.path.basename(video))[0]
  
  labeled_video = video_name+'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered_id_labeled.mp4'

  behavior_file = open(output_dir + video_name + '_forestv5_behavior.pickle', 'rb')
  behavior = pickle.load(behavior_file)
  behavior_file.close()

  cap = cv2.VideoCapture(project_path + 'videos/' + labeled_video)

  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")

  while cap.isOpened():
    ret,frame = cap.read()
    print(frame.shape)
    frame_width = int(frame.shape[1])
    frame_height = int(frame.shape[0])
    break

  output_video = output_dir + video_name + '_forest_behavior_video.mp4'

  out = cv2.VideoWriter(output_video,cv2.VideoWriter_fourcc(*'MP4V'), 30, (frame_width,frame_height))

  count = 0
  while cap.isOpened():
    ret,frame = cap.read()
    current_behavior = behavior["behavior"][count]
    text_color = (0,0,0)
    if current_behavior == 'male_side':
      text_color = (0, 0, 150)
    elif current_behavior == 'mount':
      text_color = (0, 0, 255)
    elif current_behavior == 'female_side':
      text_color = (50, 0, 0)
    elif current_behavior == 'forest_mount':
      text_color = (0, 255, 0)
    elif current_behavior == 'unclear':
      text_color = (0, 255, 0)
    else:
      print(current_behavior)
      print(count)

    if current_behavior == 'forest_mount' or current_behavior == 'unclear':
      cv2.putText(frame, str(current_behavior), (25, 25), cv2.FONT_HERSHEY_DUPLEX, 1, text_color, 2)
      cv2.putText(frame, str(count), (25, frame_height), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2)
      out.write(frame)

    count = count + 1
    if count > end_frame:
      break
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  cap.release()
  out.release()

(166, 286, 3)
nan
2018
nan
2019
nan
2020
nan
2021
nan
2094
nan
3603
nan
3604
nan
3605
nan
3606
nan
4259
nan
4260
nan
4261
